# Load data

In [1]:
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')
    path_to_file = '/content/drive/MyDrive/DSA4212_Assignment2/datasets'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/drive
/content/drive/MyDrive/DSA4212_Assignment2/datasets
/content/drive/.shortcut-targets-by-id/1sKnwfHp4YEyYFgL7Arxyl1vY9sS5U5dH/DSA4212_Assignment2/datasets


In [117]:
!pip install recsys

     |████████████████████████████████| 1.4 MB 4.2 MB/s 
  Created wheel for recsys: filename=recsys-0.0.4-cp37-cp37m-linux_x86_64.whl size=750799 sha256=aef1add45bf3ad6fc7edd6411701703976506c36dc789761f70caee1794188ad
  Stored in directory: /root/.cache/pip/wheels/26/12/83/61b5f75590ea53e746469255bc414104fadac60c3dcb4f9246
Successfully built recsys


In [120]:
!pip install preprocessing

     |████████████████████████████████| 349 kB 3.9 MB/s 
     |████████████████████████████████| 1.2 MB 40.5 MB/s 
     |████████████████████████████████| 1.4 MB 34.3 MB/s 
  Created wheel for nltk: filename=nltk-3.2.4-py3-none-any.whl size=1367717 sha256=288bd9953e4a3ac6bdf4f2278357b072f607ec753bd632a853ae5b7c972514c5
  Stored in directory: /root/.cache/pip/wheels/90/5e/9e/4cb46185f2a16c60e6fc524372ba7fef89ce3347734c8798b6
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [122]:
from sklearn.datasets import dump_svmlight_file
import numpy as np
import pandas as pd
import os
import urllib
import zipfile
from sklearn.model_selection import train_test_split
import shutil
 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from recsys import * 
from preprocessing import * 


%matplotlib inline

ratings = pd.read_table("ratings.dat",sep = '::',header=None,engine='python',names=['user_id','movie_id',"rating","timestamp"])
users = pd.read_table("users.dat",sep = '::',header=None,engine='python',names =['user_id','gender','age','occupation','zipcode'])
movies = pd.read_table('movies.dat', sep='::', header=None, engine='python', encoding='ISO-8859-1').to_numpy()


#movies.columns = ['movie_id','title','genres']

In [123]:
movies

array([[1, 'Toy Story (1995)', "Animation|Children's|Comedy"],
       [2, 'Jumanji (1995)', "Adventure|Children's|Fantasy"],
       [3, 'Grumpier Old Men (1995)', 'Comedy|Romance'],
       ...,
       [3950, 'Tigerland (2000)', 'Drama'],
       [3951, 'Two Family House (2000)', 'Drama'],
       [3952, 'Contender, The (2000)', 'Drama|Thriller']], dtype=object)

In [124]:
dfs = []
for i in movies:
  movie=[]
  movie.append(i[0])
  movie.append(i[1])
  movie.append(i[2])
  dfs.append(movie)

In [125]:
movies = pd.DataFrame(dfs, columns=['movie_id','movie_title','genre'])


In [138]:
!pip install lightfm

     |████████████████████████████████| 310 kB 4.4 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705356 sha256=2cb8b7af2ada1a80832db291e4c4236c47e18eb1e91db4b041b8e087c422ac5f
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [139]:
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity

In [129]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions
def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict
    
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [133]:
interactions = create_interaction_matrix(df = ratings, user_col = 'user_id', item_col = 'movie_id', rating_col = 'rating')
interactions.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
user_dict = create_user_dict(interactions=interactions)
movies_dict = create_item_dict(df = movies, id_col = 'movie_id', name_col = 'movie_title')

In [135]:
def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    '''
    Function to run matrix-factorization algorithm
    Required Input -
        - interactions = dataset create by create_interaction_matrix
        - n_components = number of embeddings you want to create to define Item and user
        - loss = loss function other options are logistic, brp
        - epoch = number of epochs to run 
        - n_jobs = number of cores used for execution 
    Expected Output  -
        Model - Trained model
    '''
    x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

In [140]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [141]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [143]:
rec_list = sample_recommendation_user(model = mf_model, interactions = interactions, user_id = 1, user_dict = user_dict, item_dict = movies_dict, threshold = 4, nrec_items = 10, show = True)

Known Likes:
1- Awakenings (1990)
2- Christmas Story, A (1983)
3- Bug's Life, A (1998)
4- Saving Private Ryan (1998)
5- Rain Man (1988)
6- Last Days of Disco, The (1998)
7- Ben-Hur (1959)
8- Back to the Future (1985)
9- One Flew Over the Cuckoo's Nest (1975)
10- Sound of Music, The (1965)
11- Dumbo (1941)
12- Mary Poppins (1964)
13- Cinderella (1950)
14- Beauty and the Beast (1991)
15- Schindler's List (1993)
16- Apollo 13 (1995)
17- Pocahontas (1995)
18- Toy Story (1995)

 Recommended Items:
1- Lion King, The (1994)
2- E.T. the Extra-Terrestrial (1982)
3- Wizard of Oz, The (1939)
4- Aladdin (1992)
5- Snow White and the Seven Dwarfs (1937)
6- Little Mermaid, The (1989)
7- American Beauty (1999)
8- Fantasia (1940)
9- Babe (1995)
10- Shawshank Redemption, The (1994)


In [144]:
def sample_recommendation_item(model,interactions,item_id,user_dict,item_dict,number_of_user):
    '''
    Funnction to produce a list of top N interested users for a given item
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - item_id = item ID for which we need to generate recommended users
        - user_dict =  Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - number_of_user = Number of users needed as an output
    Expected Output -
        - user_list = List of recommended users 
    '''
    n_users, n_items = interactions.shape
    x = np.array(interactions.columns)
    scores = pd.Series(model.predict(np.arange(n_users), np.repeat(x.searchsorted(item_id),n_users)))
    user_list = list(interactions.index[scores.sort_values(ascending=False).head(number_of_user).index])
    return user_list 


In [145]:
sample_recommendation_item(model = mf_model, interactions = interactions, item_id = 1, user_dict = user_dict, item_dict = movies_dict, number_of_user = 15)

[2758,
 1857,
 852,
 3604,
 5975,
 2108,
 5028,
 213,
 606,
 2799,
 1493,
 4300,
 6015,
 1728,
 4864]

In [146]:
def create_item_emdedding_distance_matrix(model,interactions):
    '''
    Function to create item-item distance embedding matrix
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
    Expected Output -
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
    '''
    df_item_norm_sparse = sparse.csr_matrix(model.item_embeddings)
    similarities = cosine_similarity(df_item_norm_sparse)
    item_emdedding_distance_matrix = pd.DataFrame(similarities)
    item_emdedding_distance_matrix.columns = interactions.columns
    item_emdedding_distance_matrix.index = interactions.columns
    return item_emdedding_distance_matrix

In [147]:
## Creating item-item distance matrix
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model, interactions = interactions)
## Checking item embedding distance matrix
item_item_dist.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.509982,-0.155090,-0.182740,-0.014191,0.207568,0.114690,0.274060,-0.188546,0.316740,...,-0.098871,-0.325133,0.078590,-0.163239,-0.362660,0.201064,-0.063095,-0.167513,-0.295620,-0.001050
2,0.509982,1.000000,-0.019189,-0.185981,-0.010543,-0.063481,-0.011018,0.524368,0.145651,0.437477,...,-0.357129,-0.146844,-0.007087,-0.131883,-0.389344,-0.083826,-0.121084,-0.212906,-0.303349,-0.102829
3,-0.155090,-0.019189,1.000000,0.311971,0.339132,-0.279166,0.399067,0.138481,0.308913,-0.028153,...,0.092641,0.244764,0.255500,0.257339,0.259504,0.213615,0.066777,0.008416,-0.014368,-0.017422
4,-0.182740,-0.185981,0.311971,1.000000,0.673202,-0.191438,0.529022,-0.118473,0.038392,-0.076062,...,-0.054721,0.082684,-0.178886,-0.254113,-0.284231,-0.142930,-0.164154,-0.311542,0.049710,-0.223581
5,-0.014191,-0.010543,0.339132,0.673202,1.000000,-0.209030,0.573458,0.076995,0.143766,0.030478,...,-0.239131,0.080135,-0.009200,-0.225475,-0.422288,-0.161285,-0.415418,-0.352640,-0.200527,-0.308915


In [149]:
def item_item_recommendation(item_emdedding_distance_matrix, item_id, 
                             item_dict, n_items = 10, show = True):
    '''
    Function to create item-item recommendation
    Required Input - 
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
        - item_id  = item ID for which we need to generate recommended items
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - n_items = Number of items needed as an output
    Expected Output -
        - recommended_items = List of recommended items
    '''
    recommended_items = list(pd.Series(item_emdedding_distance_matrix.loc[item_id,:]. \
                                  sort_values(ascending = False).head(n_items+1). \
                                  index[1:n_items+1]))
    if show == True:
        print("Item of interest :{0}".format(item_dict[item_id]))
        print("Item similar to the above item:")
        counter = 1
        for i in recommended_items:
            print(str(counter) + '- ' +  item_dict[i])
            counter+=1
    return recommended_items

In [152]:
## Calling 10 recommended items for item id 
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist, item_id = 1, item_dict = movies_dict, n_items = 10)

Item of interest :Toy Story (1995)
Item similar to the above item:
1- Aladdin (1992)
2- Toy Story 2 (1999)
3- Bug's Life, A (1998)
4- Pleasantville (1998)
5- Babe (1995)
6- Babe: Pig in the City (1998)
7- Mask, The (1994)
8- Muppet Movie, The (1979)
9- Wrong Trousers, The (1993)
10- Close Shave, A (1995)
